In [2]:
import os
import pickle
import random
import time
import PIL
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

In [12]:
from keras import Input, Model
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.layers import Dense, LeakyReLU, BatchNormalization, ReLU, Reshape, UpSampling2D, Conv2D, Activation, concatenate, Flatten, Lambda, Concatenate, ZeroPadding2D
from keras.optimizers import Adam
from keras_preprocessing.image import ImageDataGenerator

In [4]:
def generate_c(x):
    
    stddev = K.exp(log_sigma)
    epsilon = K.random_normal(shape = K.constant((mean.shape[1],), dtype = 'int32'))
    c = stddev * epsilon + mean
    
    return c

In [6]:
def build_ca_model():
    
    input_layer = Input(shape = (1024,))
    x = Dense(256)(input_layer)
    mean_logsigma = LeakyReLU(alpha = 0.2)(x)
    
    c = Lambda(generate_c)(mean_logsigma)
    
    model = Model(inputs = [input_layer], outputs = [c])
    
    return model

In [7]:
def build_gen_1():
    
    input_layer = Input(shape = (1024,))
    x = Dense(256)(input_layer)
    mean_logsigma = LeakyReLU(alpha = 0.2)(x)
    
    c = Lambda(generate_c)(mean_logsigma)
    
    
    input_layer2 = Input(shape = (100,))
    
    gen_input = Concatenate(axis = 1)([c, input_layer2])
    
    x = Dense(128 * 8 * 4* 4, use_bias = False)(gen_input)
    x = ReLU()(x)
    
    x = Reshape((4, 4, 128 * 8), input_shape = (128 * 8 * 4* 4,))(x)
    
    x = UpSampling2D(size = (2, 2))(x)
    
    x = Conv2D(512, kernel_size = 3, padding = 'same', strides = 1, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = UpSampling2D(size = (2, 2))(x)
    
    x = Conv2D(256, kernel_size = 3, padding = 'same', strides = 1, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = UpSampling2D(size = (2, 2))(x)
    
    x = Conv2D(128, kernel_size = 3, padding = 'same', strides = 1, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = UpSampling2D(size = (2, 2))(x)
    
    x = Conv2D(64, kernel_size = 3, padding = 'same', strides = 1, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    
    x = Conv2D(3, kernel_size = 3, padding = 'same', strides = 1, use_bias = False)(x)
    x = Activation(activation = 'tanh')(x)
    
    stage_1_gan = Model(inputs = [input_layer, input_layer2], outputs = [x, mean_logsigma])
    
    return stage_1_gan
    

In [9]:
def build_disc_1():
    
    input_layer = Input(shape = (64, 64, 3))
    
    x = Conv2D(64, (4, 4), padding = 'same', strides = 2, input_shape = (64, 64, 3), use_bias = False)(input_layer)
    x = LeakyReLU(alpha = 0.2)(x)
    
    x = Conv2D(128, (4, 4), padding = 'same', strides = 2, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.2)(x)
    
    x = Conv2D(256, (4, 4), padding = 'same', strides = 2, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.2)(x)
    
    x = Conv2D(512, (4, 4), padding = 'same', strides = 2, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.2)(x)
    
    input_layer2 = Input(shape = (4, 4, 128))
    
    merged_input = concatenate([x, input_layer2])
    
    x2 = Conv2D(64 * 8, kernel_size = 1, padding = 'same', strides = 1)(merged_input)
    x2 = BatchNormalization()(x2)
    x2 = LeakyReLU(alpha = 0.2)(x2)
    
    x2 = Flatten()(x2)
    
    x2 = Dense(1)(x2)
    x2 = Activation('sigmoid')(x2)
    
    stage_1_disc = Model(inputs = [input_layer, input_layer2], outputs = [x2])
    
    return stage_1_disc

In [11]:
def build_adversarial_model(gen_model, disc_model):
    
    input_layer = Input(shape = (1024,))
    input_layer2 = Input(shape = (100,))
    input_layer3 = Input(shape = (4, 4, 128))
    
    x, mean_logsigma = gen_model([input_layer, input_layer2])
    
    disc_model.trainable = False
    
    valid = disc_model([x, input_layer3])
    
    model = Model(inputs = [input_layer, input_layer2, input_layer3], outputs = [valid, mean_logsigma])
    
    return model

In [13]:
def joint_block(inputs):
    
    c = inputs[0]
    x = inputs[1]
    
    c = K.expand_dims(c, axis = 1)
    c = K.expand_dims(c, axis = 1)
    c = K.tile(c, [1, 16, 16, 1])
    c = K.concatenate([c, x], axis = 3)
    
    return c

In [16]:
def residual_block(inputs):
    
    x = Conv2D(128 * 4, kernel_size = (3, 3), padding = 'same', strides = 1)(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(128 * 4, kernel_size = (3, 3), padding = 'same', strides = 1)(x)
    x = BatchNormalization()(x)
    
    x = add([x, inputs])
    x = ReLU()(x)
    
    return x

In [17]:
def build_gen_2():
    
    input_layer = Input(shape = (1024,))
    input_lr_images = Input(shape = (64, 64, 3))
    
    ca = Dense(256)(input_layer)
    mean_logsigma = LeakyReLU(alpha = 0.2)(ca)
    c = Lambda(generate_c)(mean_logsigma)
    
    x = ZeroPadding2D(padding = (1, 1))(input_lr_images)
    x = Conv2D(128, kernel_size = (3, 3), strides = 1, use_bias = False)(x)
    x = ReLU()(x)
    
    x = ZeroPadding2D(padding = (1, 1))(x)
    x = Conv2D(256, kernel_size = (4, 4), strides = 1, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = ZeroPadding2D(padding = (1, 1))(x)
    x = Conv2D(512, kernel_size = (4, 4), strides = 1, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    c_code = Lambda(joint_block)([c,x])
    
    x = ZeroPadding2D(padding = (1, 1))(c_code)
    x = Conv2D(512, kernel_size = (3, 3), strides = 1, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = residual_block(x)
    x = residual_block(x)
    x = residual_block(x)
    x = residual_block(x)
    
    x = UpSampling2D(sixe = (2, 2))(x)
    x = Conv2D(512, kernel_size = 3, padding = 'same', strides = 1, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = UpSampling2D(sixe = (2, 2))(x)
    x = Conv2D(256, kernel_size = 3, padding = 'same', strides = 1, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = UpSampling2D(sixe = (2, 2))(x)
    x = Conv2D(128, kernel_size = 3, padding = 'same', strides = 1, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = UpSampling2D(sixe = (2, 2))(x)
    x = Conv2D(64, kernel_size = 3, padding = 'same', strides = 1, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(3, kernel_size = 3, padding = 'same', strides = 1, use_bias = False)(x)
    x = Activation(activation = 'tanh')(x)
    
    model = Model(inputs = [input_layer, input_lr_images], outputs = [x, mean_logsigma])
    
    return model

In [18]:
def build_disc_2():
    
    input_layer = Input(shape = (256, 256, 3))
    
    x = Conv2D(64, (4, 4), padding = 'same', strides = 2, input_shape = (256, 256, 3), use_bias = False)(input_layer)
    x = LeakyReLU(alpha = 0.2)(x)
    
    x = Conv2D(128, (4, 4), padding = 'same', strides = 2, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.2)(x)
    
    x = Conv2D(256, (4, 4), padding = 'same', strides = 2, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.2)(x)
    
    x = Conv2D(512, (4, 4), padding = 'same', strides = 2, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.2)(x)
    
    x = Conv2D(1024, (4, 4), padding = 'same', strides = 2, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.2)(x)
    
    x = Conv2D(2048, (4, 4), padding = 'same', strides = 2, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.2)(x)
    
    x = Conv2D(1024, (1, 1), padding = 'same', strides = 1, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.2)(x)
    
    x = Conv2D(512, (1, 1), padding = 'same', strides = 1, use_bias = False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.2)(x)
    
    x2 = Conv2D(128, (1, 1), padding = 'same', strides = 1, use_bias = False)(x)
    x2 = BatchNormalization()(x2)
    x2 = LeakyReLU(alpha = 0.2)(x2)
    
    x2 = Conv2D(128, (3, 3), padding = 'same', strides = 1, use_bias = False)(x2)
    x2 = BatchNormalization()(x2)
    x2 = LeakyReLU(alpha = 0.2)(x2)
    
    x2 = Conv2D(512, (3, 3), padding = 'same', strides = 1, use_bias = False)(x2)
    x2 = LeakyReLU(alpha = 0.2)(x2)
    
    added_x = add([x, x2])
    added_x = LeakyReLU(alpha = 0.2)(added_x)
    
    input_layer2 = Input(shape = (4, 4, 128))
    
    merged_input = concatenate([added_x, input_layer2])
    
    x3 = Conv2D(64 * 8, kernel_size = 1, padding = 'same', strides = 1)(mergged_input)
    x3 = BatchNormalization()(x3)
    x3 = LeakyReLU(alpha = 0.2)(x3)
    x3 = Flatten()(x3)
    x3 = Dense(1)(x3)
    x3 = Activation('sigmoid')(x3)
    
    stage2_disc = Model(inputs = [input_layer, input_layer2], outputs = [x3])
    
    return stage2_disc

In [19]:
def load_class_ids(class_info_file_path):
    
    with open(class_info_file_path, 'rb') as f:
        class_ids = pickle.load(f, encodings  = 'latin1')
        
    return class_ids

In [20]:
def load_filename(file_names_file_path):
    
    with open(file_names_file_path, 'rb') as f:
        filenames = pickle.load(f, encoding = 'latin1')
        
    return filenames

In [21]:
def load_embeddings(embeddings_file_path):
    
    with open(embeddings_file_path, 'rb') as f:
        embeddings = pickle.load(f, encoding = 'latin1')
        embeddings = np.array(embeddings)
        
        print("Embeddings: ", embeddings.shape)
        
    return embeddings

In [ ]:
def load_boxes(dataset_dir):
    
    bounding_boxes_path = os.path.join(dataset_dir, 'images.txt')
    
    df_bounding_boxes = pd.read_csv(bounding_boxes_path, delim_whitespace = True, header = None).astype(int)
    
    df_file_names = pd.read_csv(file_paths_path, delim_whitespace = True, header = None)
    
    file_names = df_file_names[1].tolist()
    